In [26]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import minimize
from scipy.stats import norm

In [2]:
address = 'C:\\Users\\moham\\My Drive\\Projects\\Structural Econometrics Projects\\Topics in Advanced Econometrics\\2- Logit'

In [3]:
commute_data = pd.read_csv(address + '\\commute_multinomial.csv')

# Problem 1: Maximum Likelihood Estimation

Again, the problem is for commute. Each agent has four alternatives to choose from. The utility is as follows:

$$
V_{nj} = \beta C_{nj} + \gamma T_{nj}
$$
Therefore, given a multinomial logit model, the log-likelihood function is as follows:
$$
\ln L(\theta \mid y, \boldsymbol{X})=\sum_{n=1}^N \sum_{i=1}^J y_{n i} \ln \left[\frac{e^{\beta C_{n,i} + \gamma T_{n,i}}}{\sum_j e^{\beta C_{n,j} + \gamma T_{n,j}}}\right]
$$

So, we will define a function that, given the parameters and data as inputs, and solve for the loglikelihood function. But, first, we need to prepare the data.

In [4]:
# List of modes
modes = ['bike', 'bus', 'car', 'walk']

# Reshape the DataFrame
rows = []
for index, row in commute_data.iterrows():
    for mode in modes:
        rows.append({
            'id': row['id'],
            'mode': mode,
            'time': row[f'time.{mode}'],
            'cost': row[f'cost.{mode}'],
            'choice': 1 if row['mode'] == mode else 0
        })

commute_long = pd.DataFrame(rows)

In [5]:
commute_long.head(4)

,id,mode,time,cost,choice
0,1,bike,20,0.00,0
1,1,bus,20,0.00,1
2,1,car,16,0.82,0
3,1,walk,55,0.00,0


In [6]:
def loglikelihood(params, data):
    # Create a copy of the input DataFrame to avoid modifying the original one
    df = data.copy()
    # Independent variables as a NumPy array
    X = df[['time', 'cost']]
    # Calculating the exponential utility
    df['exp_Util'] = np.exp(np.dot(X, params))
    # Summing exp_Util for each id
    df['sum_exp_Util'] = df.groupby('id')['exp_Util'].transform('sum')
    # Probability calculation
    df['prob'] = df['exp_Util'] / df['sum_exp_Util']
    # Calculating the log-likelihood
    loglikelihood_value = np.sum(df['choice'] * np.log(df['prob']))
    # Return negative log-likelihood (for minimization)
    return -loglikelihood_value

Now, we will find the optimal parameters.

In [24]:
initial_params = np.zeros(2)
result = minimize(loglikelihood, 
                  initial_params, 
                  args=(commute_long[['id', 'choice', 'time', 'cost']],), 
                  method='BFGS', 
                  options={'disp': True})
parameters = result.x

Optimization terminated successfully.
         Current function value: 1219.844944
         Iterations: 11
         Function evaluations: 51
         Gradient evaluations: 17


Now, from MLE, we remember that the asymptotic distribution is as follows:

$$
\widehat{\boldsymbol{\theta}} \stackrel{a}{\sim} \mathcal{N}\left(\boldsymbol{\theta}_0, I\left(\boldsymbol{\theta}_0\right)^{-1}\right) \,\, ,\,\, I\left(\boldsymbol{\theta}_0\right)=-E_0\left[\frac{\partial^2 \ln L\left(\boldsymbol{\theta}_0\right)}{\partial \boldsymbol{\theta}_0 \partial \boldsymbol{\theta}_0^{\prime}}\right]
$$
Then:
$$
\widehat{\operatorname{Var}}(\widehat{\boldsymbol{\theta}})=\left\{-\left.\frac{\partial^2 \ln L(\boldsymbol{\theta})}{\partial \boldsymbol{\theta} \partial \boldsymbol{\theta}^{\prime}}\right|_{\boldsymbol{\theta}=\widehat{\boldsymbol{\theta}}}\right\}^{-1}
$$

In [34]:
std = np.sqrt(result.hess_inv.diagonal())
z_stat = parameters / std
p_values = 2 * (1 - norm.cdf(abs(z_stat)))
summary_table = pd.DataFrame({
    'Coefficient': parameters,
    'Standard Error': std,
    'Z-statistic': z_stat,
    'P-value': p_values
})
print(summary_table.round(3).to_string(index=False))

 Coefficient  Standard Error  Z-statistic  P-value
      -0.126           0.009      -14.191      0.0
      -1.002           0.171       -5.854      0.0
